# California Current Shapefile Map

This notebook downloads the California Current Large Marine Ecosystem (LME #3) shapefile from Marine Regions and creates a map.

**Objectives:**
- Download California Current shapefile from marineregions.org
- Load and inspect the shapefile
- Create a map showing the California Current boundary
- Save the map to figures directory

## 1. Setup and Imports

In [ ]:
# Import libraries
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import requests
from pathlib import Path

%matplotlib inline

In [ ]:
# Set up paths
data_dir = Path('../data/raw/shapefiles')
figures_dir = Path('../figures')

# Ensure directories exist
data_dir.mkdir(parents=True, exist_ok=True)
figures_dir.mkdir(parents=True, exist_ok=True)

## 2. Download California Current Shapefile

The California Current is designated as Large Marine Ecosystem (LME) #3.
We'll download it from the Marine Regions database.

In [ ]:
# California Current LME via Marine Regions WFS (Web Feature Service)
# The California Current is Large Marine Ecosystem #3

print("Downloading California Current (LME #3) from Marine Regions WFS...")

# WFS endpoint
url = "https://geo.vliz.be/geoserver/MarineRegions/wfs"

# Specify parameters for WFS request
params = dict(
    service="WFS",
    version="2.0.0",
    request="GetFeature",
    typeName="MarineRegions:lme",
    outputFormat="application/json",
    cql_filter="lme_number=3"  # Filter for California Current (LME #3)
)

print(f"WFS endpoint: {url}")
print(f"Requesting: {params['typeName']}")
print(f"Filter: {params['cql_filter']}")

# Fetch data from WFS using requests
r = requests.get(url, params=params)
r.raise_for_status()  # Raise error if request failed

# Create GeoDataFrame from geojson
# Marine Regions data is in WGS84 (EPSG:4326)
california_current = gpd.GeoDataFrame.from_features(
    r.json(),  # requests.json() parses the JSON response
    crs="EPSG:4326"
)

print(f"\nDownload complete!")
print(f"Downloaded {len(california_current)} feature(s)")

# Save to local shapefile for future use
shapefile_path = data_dir / 'california_current_lme.shp'
california_current.to_file(shapefile_path)
print(f"Saved to: {shapefile_path}")

## 3. Load and Inspect Shapefile

In [ ]:
# Find the main shapefile
shapefile = list(data_dir.glob('*.shp'))[0]
print(f"Loading shapefile: {shapefile}")

# Load all LMEs
lme_all = gpd.read_file(shapefile)
print(f"\nLoaded {len(lme_all)} LME polygons")
print(f"CRS: {lme_all.crs}")

In [ ]:
# Inspect the downloaded California Current LME data
print(f"Loaded {len(california_current)} LME polygon(s)")
print(f"CRS: {california_current.crs}")
print(f"\nColumns: {california_current.columns.tolist()}")

In [ ]:
# Display California Current LME information
california_current.head()

# Display California Current LME details
print("California Current LME:")
if 'lme_name' in california_current.columns:
    print(f"  Name: {california_current['lme_name'].iloc[0]}")
if 'lme_number' in california_current.columns:
    print(f"  LME Number: {california_current['lme_number'].iloc[0]}")
print(f"  Geometry type: {california_current.geometry.type.iloc[0]}")
print(f"  Bounds: {california_current.total_bounds}")

In [ ]:
# Create figure with cartopy projection
fig, ax = plt.subplots(figsize=(12, 12),
                       subplot_kw={'projection': ccrs.PlateCarree()})

# Plot California Current boundary
california_current.plot(ax=ax, 
                       facecolor='lightblue', 
                       edgecolor='darkblue', 
                       alpha=0.5,
                       linewidth=2)

# Add geographic features
ax.add_feature(cfeature.LAND, facecolor='lightgray', edgecolor='black')
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5)
ax.add_feature(cfeature.OCEAN, facecolor='lightcyan')

# Add gridlines with labels
gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5, color='gray')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

# Set extent to focus on California Current region
# Approximate bounds: 20-50°N, 115-135°W
ax.set_extent([-140, -110, 15, 55], crs=ccrs.PlateCarree())

# Add title
ax.set_title('California Current Large Marine Ecosystem (LME #3)', 
            fontsize=16, fontweight='bold', pad=20)

# Add text annotation
ax.text(0.02, 0.98, 'Source: Marine Regions (marineregions.org)',
       transform=ax.transAxes, fontsize=8, verticalalignment='top',
       bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.tight_layout()

# Save figure
output_file = figures_dir / 'california_current_map.png'
plt.savefig(output_file, dpi=300, bbox_inches='tight')
print(f"\nFigure saved to: {output_file}")

plt.show()